# Natural Language Processing

## Exercise Sheet 4

In [1]:
#imports for all exercises
import pprint
from nltk import corpus, FreqDist, memoize
from nltk.corpus import wordnet as wn
import timeit

### Exercise 1

Write a program to initialize a two-dimensional array of sets called `word_vowels` and process a list of words, adding each word to `word_vowels[l][v]` where $l$ is the length of the word and $v$ is the number of vowels it contains. Test your program with a 10x10-array and the list `['Alice', 'hat', 'heute', 'ihren', 'freien', 'Tag']`.

In [2]:
def word_vowels(words, dim):
    # creates array with dim x dim sets
    word_vowels =  [ [ set() for _ in range(dim) ] for _ in range(dim) ]
    for word in words:
        l = len(word)
        # number of vowels in word
        v = sum( 1 for c in word if c in 'aeiou' )
        word_vowels[l][v].add(word)
    return word_vowels

word_vowels(['Alice', 'hat', 'heute', 'ihren', 'freien', 'Tag'], 10)

[[set(), set(), set(), set(), set(), set(), set(), set(), set(), set()],
 [set(), set(), set(), set(), set(), set(), set(), set(), set(), set()],
 [set(), set(), set(), set(), set(), set(), set(), set(), set(), set()],
 [set(),
  {'Tag', 'hat'},
  set(),
  set(),
  set(),
  set(),
  set(),
  set(),
  set(),
  set()],
 [set(), set(), set(), set(), set(), set(), set(), set(), set(), set()],
 [set(),
  set(),
  {'Alice', 'ihren'},
  {'heute'},
  set(),
  set(),
  set(),
  set(),
  set(),
  set()],
 [set(), set(), set(), {'freien'}, set(), set(), set(), set(), set(), set()],
 [set(), set(), set(), set(), set(), set(), set(), set(), set(), set()],
 [set(), set(), set(), set(), set(), set(), set(), set(), set(), set()],
 [set(), set(), set(), set(), set(), set(), set(), set(), set(), set()]]

### Exercise 2

Write a program that prints all words that only appear in the last 10\% of a text. Test your code with the file `'shakespeare-macbeth.txt'` from the Gutenberg Corpus.


In [3]:
def last_ten_percent_words(text):
    cutoff = int(len(text) * 0.9)
    # set of words used in first 90% of text -> case insensitive and only alphabetical
    first_part = set( w.lower() for w in text[: cutoff] if w.isalpha() )
    # set of words used in last 10% of text
    last_part = set( w.lower() for w in text[cutoff :] if w.isalpha() )
    # set of words ONLY used in last part (not present in first)
    only_last_part = set( w for w in last_part if not w in first_part )
    return sorted(only_last_part)

shakespeare = corpus.gutenberg.words('shakespeare-macbeth.txt')
last_ten_percent_words(shakespeare)

['abhorred',
 'absent',
 'aduance',
 'aduantage',
 'ague',
 'alarums',
 'alowd',
 'appeare',
 'approaches',
 'arbitrate',
 'army',
 'arriu',
 'auouches',
 'auoyded',
 'backward',
 'baited',
 'bane',
 'battell',
 'bear',
 'beard',
 'beaten',
 'beest',
 'befor',
 'beleeu',
 'birnane',
 'bloodier',
 'bough',
 'boughes',
 'brandish',
 'brauely',
 'breefe',
 'bruited',
 'butcher',
 'calling',
 'candle',
 'censures',
 'chambers',
 'cheapely',
 'childrens',
 'clamorous',
 'clatter',
 'com',
 'compast',
 'confident',
 'constrained',
 'continued',
 'cool',
 'cosins',
 'cow',
 'creepes',
 'crests',
 'cyme',
 'darefull',
 'debt',
 'decision',
 'direnesse',
 'discouery',
 'drugge',
 'dusty',
 'earles',
 'either',
 'endure',
 'equiuocation',
 'erre',
 'euent',
 'exil',
 'expence',
 'fairer',
 'familiar',
 'famine',
 'fearefull',
 'fiends',
 'fighting',
 'finis',
 'flying',
 'forc',
 'forgot',
 'gaze',
 'gently',
 'ghosts',
 'ginne',
 'groue',
 'haires',
 'harbingers',
 'hardly',
 'harnesse',
 'hate

### Exercise 3

Write a program that takes a sentence expressed as a single string, splits it and counts up the words. Get it to print out each word and the word's frequency, one per line, in alphabetical order. Test it with the sentence: `'das ist heute wieder einmal wirklich ein sehr schöner tag das kann ich dir wieder einmal sagen'`.


In [4]:
def print_word_freq(sentence):
    words = sentence.split()
    fdist = FreqDist(words)
    # creates sorted set of tuples with (word, corresponding frequency)
    word_and_freq = sorted(set( (word, fdist[word]) for word in words ))
    for wf in word_and_freq:
        print(f'{wf[0]}: {wf[1]}')

sentence = 'das ist heute wieder einmal wirklich ein sehr schöner tag das kann ich dir wieder einmal sagen'

print_word_freq(sentence)

das: 2
dir: 1
ein: 1
einmal: 2
heute: 1
ich: 1
ist: 1
kann: 1
sagen: 1
schöner: 1
sehr: 1
tag: 1
wieder: 2
wirklich: 1


### Exercise 4

Write a function `sort_dist(candidates, target)`. The `candidates` are a list of strings representing WordNet synset names, and `target` a synset name string. The function shall sort the `candidates` for proximity to the `target` synset using `shortest_path_distance()`. 

Test your function with `candidates=['minke_whale.n.01','orca.n.01','novel.n.01', 'tortoise.n.01']` and `target='right_whale.n.01'`.

In [5]:
def sort_dist(candidates, target):
    cand_and_syn = [ (c, wn.synset(c)) for c in candidates ]
    targ_syn = wn.synset(target)
    shortest_paths = [ (targ_syn.shortest_path_distance(cs), c) for c, cs in cand_and_syn ]
    return sorted(shortest_paths)


sort_dist(['minke_whale.n.01', 'orca.n.01', 'novel.n.01', 'tortoise.n.01'], 'right_whale.n.01')

[(3, 'minke_whale.n.01'),
 (5, 'orca.n.01'),
 (12, 'tortoise.n.01'),
 (22, 'novel.n.01')]

### Exercise 5

Write a recursive function `lookup(trie, key)` that looks up a `key` in a `trie`, and returns the value it finds. The function should cover the following cases:

a) it should return a corresponding message if the key is not included in the trie;  
b) it should return a message if the key is not unique, i.e. if there are several words for this prefix;  
c) if a word is uniquely determined by the key prefix it should be returned as result. 

Try your function for the following trie and test cases:

In [2]:
def insert(trie, key, value):
    if key:
        first, rest = key[0], key[1:]
        if first not in trie:
            trie[first] = {}
        insert(trie[first], rest, value)
    else:
        trie['value'] = value

trie = {}
insert(trie, 'chat', 'cat')
insert(trie, 'chien', 'dog')
insert(trie, 'chair', 'flesh')
insert(trie, 'chic', 'stylish')
insert(trie, 'cheval','horse')
trie = dict(trie)             
pprint.pprint(trie, width=40)

{'c': {'h': {'a': {'i': {'r': {'value': 'flesh'}},
                   't': {'value': 'cat'}},
             'e': {'v': {'a': {'l': {'value': 'horse'}}}},
             'i': {'c': {'value': 'stylish'},
                   'e': {'n': {'value': 'dog'}}}}}}


In [3]:
def lookup(trie, key):
    # recursively call function for every char of key
    # return if key is not in trie
    if key:
        try:
            return lookup(trie[key[0]], key[1:])
        except:
            return 'Given Key was not found!'
    else:
        # return if exact key is in trie 
        try:
            return trie['value']
        except:
            # skip mappings as long as they are unique
            # word could still be uniquely(!) defined by key prefix
            if len(trie) == 1:
                k = list(trie.keys())[0]
                return lookup(trie[k], '')
            # return if mapping is not unique
            else:
                return 'Given Key was not unique!'


In [4]:
print(lookup(trie, 'chat'))
print(lookup(trie, 'cha'))
print(lookup(trie, 'souris'))
print(lookup(trie, 'cheval'))
print(lookup(trie, 'che'))
print(lookup(trie, 'chev'))

cat
Given Key was not unique!
Given Key was not found!
horse
horse
horse


### Exercise 6

Write a recursive function `pp_trie` that pretty prints a trie in alphabetically sorted order by replacing common prefixes with `'-'` characters.
Test your implementation with the following example data:

**I did not finish this exercise**

### Exercise 7

The *Catalan numbers* arise in many applications of combinatorial mathematics, including the counting of parse trees. The series can be defined as follows: $C_0 = 1$, and $C_{n+1} = \sum_{i=0}^{n}C_iC_{n-i}$ for $n\geq{0}$.

Write:

a) a recursive function `cn(n)` to compute the $n$th Catalan number $C_{n}$,  
b) a corresponding function `cn2(n)` that uses dynamic programming by storing calculated solutions in a lookup table,  
c) a function `cn3(n)`, which is identical to `cn(n)` but uses a `memoize` decorator.

Test your functions first by calculating the Catalan numbers $C_0\dots C_{16}$ and then by using the `timeit` module:

In [9]:
def cn(n):
    if n < 2:
        return 1
    else: 
        sum = 0
        for i in range(n):
            sum += cn(i) * cn(n-i-1)
        return sum


In [10]:
# b)
def cn2(n):
    # init lookup with zeros
    lookup = [0] * (n+1)

    for i in range(n+1):
        # known values: cn(0) = cn(1) = 1
        if i < 2:
            lookup[i] = 1
            continue
        # rest per definition
        for j in range(i):
            lookup[i] += lookup[j] * lookup[i - j - 1]

    return lookup[n]



In [11]:
# c)
@memoize
def cn3(n):
    if n < 2:
        return 1
    else: 
        sum = 0
        for i in range(n):
            sum += cn(i) * cn(n-i-1)
        return sum

In [12]:
print(timeit.timeit("cn(16)", setup="from __main__ import cn", number=5))
print(timeit.timeit("cn2(16)", setup="from __main__ import cn2", number=5))
print(timeit.timeit("cn3(16)", setup="from __main__ import cn3", number=5))

17.219280200000014
0.00011419999998452113
3.4431681000000367
